In [4]:
import numpy as np
import pandas as pd

In [14]:
df=pd.read_csv(r'novi_datasetovi\train_test_v4.csv', parse_dates=['date'])
df.head()

,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,work_day,N Batalla de Pichincha,N Carnaval,N Cyber Monday,N Dia de Difuntos,N Dia de la Madre,N Dia del Trabajo,N Futbol,N Independencia de Cuenca,N Independencia de Guayaquil,N Navidad,N Primer dia del ano,N Terremoto Manabi,N Viernes Santo,oil_price,transactions,local_holidays,regional_holidays,national_holidays
0,2013-01-01,1,AUTOMOTIVE,0.0,0.0,Quito,Pichincha,D,13,0,0,0,0,0,0,0,0,0,0,0,1,0,0,93.14,0.0,0,0,1
1,2013-01-01,1,BABY CARE,0.0,0.0,Quito,Pichincha,D,13,0,0,0,0,0,0,0,0,0,0,0,1,0,0,93.14,0.0,0,0,1
2,2013-01-01,1,BEAUTY,0.0,0.0,Quito,Pichincha,D,13,0,0,0,0,0,0,0,0,0,0,0,1,0,0,93.14,0.0,0,0,1
3,2013-01-01,1,BEVERAGES,0.0,0.0,Quito,Pichincha,D,13,0,0,0,0,0,0,0,0,0,0,0,1,0,0,93.14,0.0,0,0,1
4,2013-01-01,1,BOOKS,0.0,0.0,Quito,Pichincha,D,13,0,0,0,0,0,0,0,0,0,0,0,1,0,0,93.14,0.0,0,0,1


In [15]:
df_wide=pd.pivot(df, index='date', columns=['store_nbr', 'family'], values='sales')
df_oil_wide=pd.pivot(df, index='date', columns=['store_nbr', 'family'], values='oil_price')

In [74]:
train=df[df['date']<'2017-08-16']
test=df[df['date']>= '2017-08-16']

In [47]:
#16, 17, 18, 19, 20, 30, 365, 730
def create_lag_features(ts: pd.Series, lags: int):
    return pd.concat(
        {
            f'y_lag_{i}': ts.shift(i)
            for i in range(1, lags + 1)
        },
        axis=1)

In [16]:
def create_lag_features2(ts: pd.Series, lags: int):
    return pd.concat(
        {
            f'y_lag_{i}': ts.shift(i)
            for i in range(lags, lags + 1)
        },
        axis=1)

In [49]:
def create_one_lag_feature(ts: pd.Series, lag: int):
    return ts.shift(lag)

In [91]:
def write_lag(lags :int):
    global df 
    
    lag_wide = pd.DataFrame()
    for col in df_wide.columns:
        lag_features = create_lag_features2(df_wide[col], lags=lags)
        lag_wide = pd.concat([lag_wide, lag_features.add_prefix(f"{col}")], axis=1)
    lag_long = lag_wide.melt(value_name=f'lag_{lags}', ignore_index=False).reset_index()
    lag_long_parsed = lag_long.copy()
    lag_long_parsed['store_nbr'] = lag_long_parsed['variable'].str.extract(r"\((\d+),")[0] 
    lag_long_parsed['family'] = lag_long_parsed['variable'].str.extract(r"'([^']*)'")
    lag_long_parsed=lag_long_parsed.drop(columns=['variable'])
    lag_long_parsed['store_nbr'] = lag_long_parsed['store_nbr'].astype(int)
    df=df.merge(lag_long_parsed, how='left', on=['date', 'store_nbr', 'family'])
    lag_long_parsed.to_csv(f'lag_{lags}.csv', index=False)

In [17]:
def write_oil_lag(lags :int):
    global df 
    
    lag_wide = pd.DataFrame()
    for col in df_oil_wide.columns:
        lag_features = create_lag_features2(df_oil_wide[col], lags=lags)
        lag_wide = pd.concat([lag_wide, lag_features.add_prefix(f"{col}")], axis=1)
    lag_long = lag_wide.melt(value_name=f'lag_{lags}_oil', ignore_index=False).reset_index()
    lag_long_parsed = lag_long.copy()
    lag_long_parsed['store_nbr'] = lag_long_parsed['variable'].str.extract(r"\((\d+),")[0] 
    lag_long_parsed['family'] = lag_long_parsed['variable'].str.extract(r"'([^']*)'")
    lag_long_parsed=lag_long_parsed.drop(columns=['variable'])
    lag_long_parsed['store_nbr'] = lag_long_parsed['store_nbr'].astype(int)
    df=df.merge(lag_long_parsed, how='left', on=['date', 'store_nbr', 'family'])
    lag_long_parsed.to_csv(f'oil_lag_{lags}.csv', index=False)

In [52]:
train_wide=pd.pivot(train, index='date', columns=['store_nbr','family'], values='sales')
train_wide.head()

store_nbr          1                                                 \
family     AUTOMOTIVE BABY CARE BEAUTY BEVERAGES BOOKS BREAD/BAKERY   
date                                                                  
2013-01-01        0.0       0.0    0.0       0.0   0.0        0.000   
2013-01-02        2.0       0.0    2.0    1091.0   0.0      470.652   
2013-01-03        3.0       0.0    0.0     919.0   0.0      310.655   
2013-01-04        3.0       0.0    3.0     953.0   0.0      198.366   
2013-01-05        5.0       0.0    3.0    1160.0   0.0      301.057   

store_nbr                                        ...        54          \
family     CELEBRATION CLEANING  DAIRY     DELI  ... MAGAZINES   MEATS   
date                                             ...                     
2013-01-01         0.0      0.0    0.0    0.000  ...       0.0   0.000   
2013-01-02         0.0   1060.0  579.0  164.069  ...       0.0  73.771   
2013-01-03         0.0    836.0  453.0  151.582  ...       0.0  50.257   
2013-01-04         0.0    827.0  460.0  131.411  ...       0.0  40.223   
2013-01-05         0.0    811.0  464.0  118.613  ...       0.0  43.431   

store_nbr                                                              \
family     PERSONAL CARE PET SUPPLIES PLAYERS AND ELECTRONICS POULTRY   
date                                                                    
2013-01-01           0.0          0.0                     0.0   0.000   
2013-01-02         228.0          0.0                     0.0  15.514   
2013-01-03         156.0          0.0                     0.0   4.313   
2013-01-04         146.0          0.0                     0.0  26.743   
2013-01-05         205.0          0.0                     0.0  31.118   

store_nbr                                                             
family     PREPARED FOODS PRODUCE SCHOOL AND OFFICE SUPPLIES SEAFOOD  
date                                                                  
2013-01-01            0.0     0.0                        0.0     0.0  
2013-01-02           61.0     0.0                        0.0     3.0  
2013-01-03            1.0     0.0                        0.0     2.0  
2013-01-04           38.0     0.0                        0.0     2.0  
2013-01-05           32.0     0.0                        0.0     1.0  

[5 rows x 1782 columns]

In [93]:
for i in [16, 17, 18, 19, 20, 30, 365, 730]:
    write_lag(i)

In [54]:
train_wide2=pd.pivot(train, index='date', columns=['store_nbr','family'], values='oil_price')
train_wide2.head()

store_nbr          1                                               \
family     AUTOMOTIVE  BABY CARE     BEAUTY  BEVERAGES      BOOKS   
date                                                                
2013-01-01  93.140000  93.140000  93.140000  93.140000  93.140000   
2013-01-02  93.140000  93.140000  93.140000  93.140000  93.140000   
2013-01-03  92.970000  92.970000  92.970000  92.970000  92.970000   
2013-01-04  93.120000  93.120000  93.120000  93.120000  93.120000   
2013-01-05  93.146667  93.146667  93.146667  93.146667  93.146667   

store_nbr                                                             ...  \
family     BREAD/BAKERY CELEBRATION   CLEANING      DAIRY       DELI  ...   
date                                                                  ...   
2013-01-01    93.140000   93.140000  93.140000  93.140000  93.140000  ...   
2013-01-02    93.140000   93.140000  93.140000  93.140000  93.140000  ...   
2013-01-03    92.970000   92.970000  92.970000  92.970000  92.970000  ...   
2013-01-04    93.120000   93.120000  93.120000  93.120000  93.120000  ...   
2013-01-05    93.146667   93.146667  93.146667  93.146667  93.146667  ...   

store_nbr          54                                        \
family      MAGAZINES      MEATS PERSONAL CARE PET SUPPLIES   
date                                                          
2013-01-01  93.140000  93.140000     93.140000    93.140000   
2013-01-02  93.140000  93.140000     93.140000    93.140000   
2013-01-03  92.970000  92.970000     92.970000    92.970000   
2013-01-04  93.120000  93.120000     93.120000    93.120000   
2013-01-05  93.146667  93.146667     93.146667    93.146667   

store_nbr                                                                \
family     PLAYERS AND ELECTRONICS    POULTRY PREPARED FOODS    PRODUCE   
date                                                                      
2013-01-01               93.140000  93.140000      93.140000  93.140000   
2013-01-02               93.140000  93.140000      93.140000  93.140000   
2013-01-03               92.970000  92.970000      92.970000  92.970000   
2013-01-04               93.120000  93.120000      93.120000  93.120000   
2013-01-05               93.146667  93.146667      93.146667  93.146667   

store_nbr                                         
family     SCHOOL AND OFFICE SUPPLIES    SEAFOOD  
date                                              
2013-01-01                  93.140000  93.140000  
2013-01-02                  93.140000  93.140000  
2013-01-03                  92.970000  92.970000  
2013-01-04                  93.120000  93.120000  
2013-01-05                  93.146667  93.146667  

[5 rows x 1782 columns]

In [18]:
for i in [1,2,3,4]:
    write_oil_lag(i)

In [98]:
df.to_csv(r'lag_so.csv',index=False)

In [99]:
#adding days after big holidays
after_holiday=pd.read_csv(r'posle_holidaya.csv', parse_dates=['date'])
after_holiday.head()

,Unnamed: 0,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,work_day,N Batalla de Pichincha,N Carnaval,N Cyber Monday,N Dia de Difuntos,N Dia de la Madre,N Dia del Trabajo,N Futbol,N Independencia de Cuenca,N Independencia de Guayaquil,N Navidad,N Primer dia del ano,N Terremoto Manabi,N Viernes Santo,oil_price,transactions,local_holidays,regional_holidays,national_holidays,month,day_of_month,day_of_year,week_of_month,week_of_year,day_of_week,year,is_wknd,quarter,is_month_start,is_month_end,is_quarter_start,is_quarter_end,is_year_start,is_year_end,date_index,season,workday,wageday,day_to_nearest_holiday,day_from_nearest_holiday
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0.0,Quito,Pichincha,D,13,0,0,0,0,0,0,0,0,0,0,0,1,0,0,93.14,0.0,0,0,1,1,1,1,1,1,2,2013,0,1,1,0,1,0,1,0,0,0,0,0,0,0
1,1,2013-01-01,1,BABY CARE,0.0,0.0,Quito,Pichincha,D,13,0,0,0,0,0,0,0,0,0,0,0,1,0,0,93.14,0.0,0,0,1,1,1,1,1,1,2,2013,0,1,1,0,1,0,1,0,0,0,0,0,0,0
2,2,2013-01-01,1,BEAUTY,0.0,0.0,Quito,Pichincha,D,13,0,0,0,0,0,0,0,0,0,0,0,1,0,0,93.14,0.0,0,0,1,1,1,1,1,1,2,2013,0,1,1,0,1,0,1,0,0,0,0,0,0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0.0,Quito,Pichincha,D,13,0,0,0,0,0,0,0,0,0,0,0,1,0,0,93.14,0.0,0,0,1,1,1,1,1,1,2,2013,0,1,1,0,1,0,1,0,0,0,0,0,0,0
4,4,2013-01-01,1,BOOKS,0.0,0.0,Quito,Pichincha,D,13,0,0,0,0,0,0,0,0,0,0,0,1,0,0,93.14,0.0,0,0,1,1,1,1,1,1,2,2013,0,1,1,0,1,0,1,0,0,0,0,0,0,0


In [100]:
after_holiday_merge=after_holiday[['date','store_nbr','family','month','day_of_month','day_of_year','week_of_month','week_of_year','day_of_week','year','is_wknd','quarter','is_month_start','is_month_end','is_quarter_start','is_quarter_end','is_year_start','is_year_end','date_index','season','workday','wageday','day_to_nearest_holiday','day_from_nearest_holiday']]

In [102]:
lag_merge=pd.read_csv(r'novi_datasetovi\lag_so.csv', parse_dates=['date'])
lag_merge.head()

,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,work_day,N Batalla de Pichincha,N Carnaval,N Cyber Monday,N Dia de Difuntos,N Dia de la Madre,N Dia del Trabajo,N Futbol,N Independencia de Cuenca,N Independencia de Guayaquil,N Navidad,N Primer dia del ano,N Terremoto Manabi,N Viernes Santo,oil_price,transactions,local_holidays,regional_holidays,national_holidays,lag_16_x,lag_17_x,lag_18_x,lag_19_x,lag_20_x,lag_30_x,lag_365_x,lag_730_x,lag_16_y,lag_17_y,lag_18_y,lag_19_y,lag_20_y,lag_30_y,lag_365_y,lag_730_y
0,2013-01-01,1,AUTOMOTIVE,0.0,0.0,Quito,Pichincha,D,13,0,0,0,0,0,0,0,0,0,0,0,1,0,0,93.14,0.0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013-01-01,1,BABY CARE,0.0,0.0,Quito,Pichincha,D,13,0,0,0,0,0,0,0,0,0,0,0,1,0,0,93.14,0.0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2013-01-01,1,BEAUTY,0.0,0.0,Quito,Pichincha,D,13,0,0,0,0,0,0,0,0,0,0,0,1,0,0,93.14,0.0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2013-01-01,1,BEVERAGES,0.0,0.0,Quito,Pichincha,D,13,0,0,0,0,0,0,0,0,0,0,0,1,0,0,93.14,0.0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2013-01-01,1,BOOKS,0.0,0.0,Quito,Pichincha,D,13,0,0,0,0,0,0,0,0,0,0,0,1,0,0,93.14,0.0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [104]:
def rename_columns(df):
    new_columns = {}
    for col in df.columns:
        if col.endswith('_x'):
            new_columns[col] = col[:-2] + '_sales'
        elif col.endswith('_y'):
            new_columns[col] = col[:-2] + '_oil'
    df.rename(columns=new_columns, inplace=True)

In [112]:
final_df=lag_merge.merge(after_holiday_merge, how='outer', on=['date','store_nbr','family'])
rename_columns(final_df) #_x is sales and _y s oil lags
final_df=final_df.drop(columns=['local_holidays','regional_holidays','national_holidays'])
final_df.head()

,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,work_day,N Batalla de Pichincha,N Carnaval,N Cyber Monday,N Dia de Difuntos,N Dia de la Madre,N Dia del Trabajo,N Futbol,N Independencia de Cuenca,N Independencia de Guayaquil,N Navidad,N Primer dia del ano,N Terremoto Manabi,N Viernes Santo,oil_price,transactions,lag_16_sales,lag_17_sales,lag_18_sales,lag_19_sales,lag_20_sales,lag_30_sales,lag_365_sales,lag_730_sales,lag_16_oil,lag_17_oil,lag_18_oil,lag_19_oil,lag_20_oil,lag_30_oil,lag_365_oil,lag_730_oil,month,day_of_month,day_of_year,week_of_month,week_of_year,day_of_week,year,is_wknd,quarter,is_month_start,is_month_end,is_quarter_start,is_quarter_end,is_year_start,is_year_end,date_index,season,workday,wageday,day_to_nearest_holiday,day_from_nearest_holiday
0,2013-01-01,1,AUTOMOTIVE,0.0,0.0,Quito,Pichincha,D,13,0,0,0,0,0,0,0,0,0,0,0,1,0,0,93.14,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1,1,1,2,2013,0,1,1,0,1,0,1,0,0,0,0,0,0,0
1,2013-01-01,1,BABY CARE,0.0,0.0,Quito,Pichincha,D,13,0,0,0,0,0,0,0,0,0,0,0,1,0,0,93.14,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1,1,1,2,2013,0,1,1,0,1,0,1,0,0,0,0,0,0,0
2,2013-01-01,1,BEAUTY,0.0,0.0,Quito,Pichincha,D,13,0,0,0,0,0,0,0,0,0,0,0,1,0,0,93.14,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1,1,1,2,2013,0,1,1,0,1,0,1,0,0,0,0,0,0,0
3,2013-01-01,1,BEVERAGES,0.0,0.0,Quito,Pichincha,D,13,0,0,0,0,0,0,0,0,0,0,0,1,0,0,93.14,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1,1,1,2,2013,0,1,1,0,1,0,1,0,0,0,0,0,0,0
4,2013-01-01,1,BOOKS,0.0,0.0,Quito,Pichincha,D,13,0,0,0,0,0,0,0,0,0,0,0,1,0,0,93.14,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1,1,1,2,2013,0,1,1,0,1,0,1,0,0,0,0,0,0,0


In [113]:
final_df.shape

(3036528, 62)

In [114]:
final_df.to_csv(r'novi_datasetovi/train_test_final.csv', index=False)